In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import folium

# READ DATA
df = pd.read_csv("C:/Users/JZ/Documents/SCHOOL/Data mining/accident_data.csv", low_memory=False)
df3 = pd.read_csv("C:/Users/JZ/Documents/SCHOOL/Data mining/traffic_data.csv", low_memory=False)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

# CLEAN DATA 
df['Time']=pd.to_datetime(df['Time'], errors='coerce')
df['Date']=pd.to_datetime(df['Date'], errors='coerce')
df = df.drop(columns=['Unnamed: 0'])
df = df.drop_duplicates(keep='first')
df = df.dropna()

# SELECT NECESSARY DATA, UNCOMMENT IF NECESSARY
df3 = df3[(df3['Lon']>=-0.3) & (df3['Lon']<=0.03)]
df3 = df3[(df3['Lat']>=51.4) & (df3['Lat']<=51.6)]
df = df[(df['Longitude']>=-0.3) & (df['Longitude']<=0.03)]
df = df[(df['Latitude']>=51.4) & (df['Latitude']<=51.6)]
df = df[(df['Accident_Severity']==1)]

#--------------------------------------------------------------------------------------
#clustering accident data
x=df.loc[:, ['Latitude', 'Longitude']]
kmeans = KMeans(n_clusters=30)
kmeans = kmeans.fit(x)
labels = kmeans.labels_
df['cluster'] = labels
plt.scatter(x['Latitude'], x['Longitude'], s=50, c=labels, marker='.',\
            cmap=plt.cm.get_cmap('jet', 30))
plt.colorbar()
plt.show()
centers = kmeans.cluster_centers_
counts = np.bincount(labels)
result = np.where(counts == np.amax(counts))
result = result[0]
print(result)
most_accidents = centers[result]
f = result.item(0) # get the int of the cluster number
print(centers[f])
print(counts, f) 
#----------------------------------------------------------------------------------------
#clustering traffic congestion data
x1=df3.loc[:, ['Lat', 'Lon']]
kmeans1 = KMeans(n_clusters=30)
kmeans1 = kmeans1.fit(x1)
labels1 = kmeans1.labels_
df3['cluster'] = labels1
plt.scatter(x1['Lat'], x1['Lon'], s=50, c=labels1, marker='.',\
            cmap=plt.cm.get_cmap('viridis', 30))
plt.colorbar()
plt.show()
centers1 = kmeans1.cluster_centers_
counts1 = np.bincount(labels1)
result1 = np.where(counts1 == np.amax(counts1))
result1 = result1[0]
print(result1)
most_accidents = centers1[result1]
f1 = result1.item(0)
print(centers1[f1])
print(counts1, f1)
#-------------------------------------------------------------------------------------------

#display folium map

location = df['Latitude'].mean(), df['Longitude'].mean()

m = folium.Map(location=location,zoom_start=6)

for i in range(0, len(centers)): #ACCIDENT DATA
    if(i==f):
        folium.CircleMarker(centers[i], radius=5, fill=True, color = 'red').add_to(m)
    else:
        folium.CircleMarker(centers[i], radius=5, fill=True, color = 'green').add_to(m)

for i in range(0, len(centers1)): #CONGESTION DATA
    if(i==f1):
        folium.CircleMarker(centers1[i], radius=5, fill=True, color = 'darkred').add_to(m)
    else:
        folium.CircleMarker(centers1[i], radius=5, fill=True, color = 'black').add_to(m)
     

    #data_heat = df[['Latitude','Longitude','Number_of_Casualties']].values.tolist()
#plugins.HeatMap(data_heat).add_to(m)

m